In [1]:
import os
# 【硬性要求】注入环境变量 (加载 .env 文件中的 OPENAI_API_KEY)
from dotenv import load_dotenv
load_dotenv(override=True)

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda

# ==========================================
# 1. 定义自定义函数 (Custom Functions)
# ==========================================
# 【科研级保姆注释】
# 旧版本：def to_titlecase(text):
# 现代写法：添加参数和返回值的类型注解 (Type Hints)。
# 为什么？在构建复杂的 Agent 时，明确的类型能帮助 LangChain 底层的 Pydantic 
# 更好地进行 Schema 验证，避免运行时的数据类型崩溃。
def to_titlecase(text: str) -> str:
    """将输入的字符串转换为标题格式 (首字母大写)"""
    return text.title()

def get_len(text: str) -> int:
    """打印文本并返回其长度"""
    # 这里的 print 是为了展示副作用 (Side Effect)，方便调试时观察数据流
    print(f"[DEBUG 拦截到的文本]: {text}") 
    return len(text)

# ==========================================
# 2. 组装 LCEL 链 (Building the Chain)
# ==========================================
prompt = ChatPromptTemplate.from_template("Give me a one-line description of {topic}")
model = ChatOpenAI(model="gpt-3.5-turbo") # 明确指定模型版本是好习惯
output_parser = StrOutputParser()

# 使用 RunnableLambda 作为适配器，将普通函数接入流水线
chain = (
    prompt 
    | model 
    | output_parser 
    | RunnableLambda(to_titlecase) 
    | RunnableLambda(get_len)
)

# ==========================================
# 3. 运行与可视化 (Execution & Visualization)
# ==========================================
if __name__ == "__main__":
    print("=== 开始执行链 ===")
    # 启动数据流
    final_output = chain.invoke({"topic": "Artificial Intelligence"})
    print(f"=== 最终输出结果 ===\n长度为: {final_output}\n")
    
    print("=== 打印执行图 (DAG) ===")
    # 【科研级保姆注释】
    # 注意：运行 print_ascii() 需要在环境中安装 grandalf 包 (`pip install grandalf`)
    # 这种 ASCII 图在服务器终端 (Terminal) 环境下调试非常有用，无需图形界面。
    try:
        chain.get_graph().print_ascii()
    except ImportError:
        print("提示：你需要安装 grandalf (`pip install grandalf`) 才能打印 ASCII 执行图。")

=== 开始执行链 ===
[DEBUG 拦截到的文本]: Artificial Intelligence Is The Simulation Of Human Intelligence Processes By Machines, Typically Computer Systems.
=== 最终输出结果 ===
长度为: 114

=== 打印执行图 (DAG) ===
    +-------------+    
    | PromptInput |    
    +-------------+    
           *           
           *           
           *           
+--------------------+ 
| ChatPromptTemplate | 
+--------------------+ 
           *           
           *           
           *           
    +------------+     
    | ChatOpenAI |     
    +------------+     
           *           
           *           
           *           
  +-----------------+  
  | StrOutputParser |  
  +-----------------+  
           *           
           *           
           *           
   +--------------+    
   | to_titlecase |    
   +--------------+    
           *           
           *           
           *           
      +---------+      
      | get_len |      
      +---------+      
           *      